### 1. 지역별 코로나 확진자 수 데이터 추출 및 전처리

In [4]:
#셀레니움, 판다스 기타 라이브러리 입력
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

#웹드라이버, 엑셀, 페이지 경로를 수정할 수 있도록 변수 설정
driver=webdriver.Chrome('./data/chromedriver.exe')
excel_location='./data'
url_region='https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%BD%94%EB%A1%9C%EB%82%98+%ED%98%84%ED%99%A9'
url_week='https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%BD%94%EB%A1%9C%EB%82%98%ED%98%84%ED%99%A9'

#menu버튼을 클릭하는 함수
def select_menu(driver):
    menu=driver.find_element_by_css_selector("section#_cs_production_type div.api_subject_bx div._content div.patients_info li.tab._tab a")
    menu.click()
    time.sleep(2)
    
#드라이버를 통해 url접속
driver.get(url_region)
time.sleep(2)    

#메뉴 클릭
select_menu(driver)

#데이터를 담을 배열 선언
regional_confirmed_data_before=[]
regional_confirmed_data=[]

#html tag를 따라서 필요한 지역과 확진자 크롤링 
regional_confirmed=driver.find_elements_by_css_selector('div#wrap div#container div#main_pack section#_cs_production_type div._content div.inner_box._map_root .local_info.type_increase')  
for confirmed in regional_confirmed:
    region=confirmed.find_elements_by_css_selector('strong.local_name')[0].text
    num=confirmed.find_elements_by_css_selector('p.population_number')[0].text
    regional_confirmed_data_before.append([region,int(num.replace(',',''))])
    
#일치하는 데이터 발생 후 생략
for v in regional_confirmed_data_before:
    if v not in regional_confirmed_data:
        regional_confirmed_data.append(v)

#column 설정 및 pandas data로 변환 
region_columns=['region','covid_num']
region_pd_data=pd.DataFrame(regional_confirmed_data,columns=region_columns)
print(region_pd_data)

   region  covid_num
0      제주       3082
1      서울     116572
2      인천      18824
3      광주       5320
4      경북       9530
5      부산      13713
6      대구      16772
7      강원       7108
8      경기     105575
9      전남       3476
10     충북       7739
11     충남      10332
12     대전       7499
13     세종       1354
14     전북       5043
15     경남      12705
16     울산       5403


### 2. 주간 국내,해외 코로나 확진자 수 데이터 추출 및 전처리

In [5]:
#마찬가지로 주간 국내외 확진자 url 접속
driver.get(url_week)
time.sleep(2)

#데이터를 담을 배열 선언
week_confirmed_data=[]

#칼럼을 첫번째 칼럼으로 설정
week_confirmed=driver.find_elements_by_css_selector('div.column')
week_confirmed[0].click()

#국내 해외 관련 경로 설정
week_data_path='div#wrap div#container div#main_pack section#_cs_production_type div.patients_info div._infect_content div._normality'

#국내유입,해외유입 칼럼명 획득
local=driver.find_elements_by_css_selector(week_data_path+' .local._y_first_title')[0].text
oversea=driver.find_elements_by_css_selector(week_data_path+' .oversea._y_second_title')[0].text
date_cnt=0

#날짜 국내확진자수 해외확진자수 데이터 크롤링
for confirmed in week_confirmed:   
    confirmed.click()
    time.sleep(2)
    week_data=driver.find_elements_by_css_selector(week_data_path+' div.turnout_cont .graph_inner .data .x_axis_value')[date_cnt].text
    local_val=driver.find_elements_by_css_selector(week_data_path+' div.turnout_graph._graph div.info_wrap .desc._y_first_value')[0].text
    oversea_val=driver.find_elements_by_css_selector(week_data_path+' div.turnout_graph._graph div.info_wrap .desc._y_second_value')[0].text
    week_confirmed_data.append([week_data,int(local_val.replace(',','')) , int(oversea_val.replace(',',''))])
    date_cnt+=1

#칼럼명 설정
week_columns=['date',local,oversea]

#주간 국내외 확진자 pands data 변환
week_pd_data=pd.DataFrame(week_confirmed_data,columns=week_columns)
print(week_pd_data)


    date  국내발생  해외유입
0  10.20  1430    11
1  10.21  1419    20
2  10.22  1487    21
3  10.23  1394    28
4  10.24  1167    23
5  10.25  1246    20
6  10.26  1930    22


### 3. 각각 데이터 엑셀 저장

In [8]:
region_pd_data.to_excel(excel_location+'\\regional_confirmed.xlsx',index=False)
week_pd_data.to_excel(excel_location+'\\week_confirmed.xlsx',index=False)

### 4. 각각 데이터 mysql 테이블 저장

###### 주의, shema 미리 준비  - SchemaName : covid

In [9]:
#라이브러리 추가
import pymysql

#mysql 스케마에 접속
conn=pymysql.connect(host='127.0.0.1',user='root',password='1234',db='covid',charset='utf8')
#커서 넘겨주고
cur=conn.cursor()

#지역별, 주간국내외 테이블 제작
cur.execute("drop table if exists regional_covid_table")
cur.execute("create table regional_covid_table (region char(2), covid_num int)")

cur.execute("drop table if exists week_covid_table")
cur.execute("create table week_covid_table (day char(5) ,local int, oversea int)")

#수행할 sql문
sql1="""insert into regional_covid_table(region,covid_num) values (%s,%s)"""
sql2="""insert into week_covid_table(day,local,oversea) values (%s, %s,%s)"""


#크롤링 데이터를 테이블에 입력
for data in regional_confirmed_data:
    cur.execute(sql1,(data[0],data[1]))
for data in week_confirmed_data:
    cur.execute(sql2,(data[0],data[1],data[2]))

#커밋하고
conn.commit()
#접속 종료
conn.close()